Question 1)Fit a logistic regression models using all of the available inputs. Identify the direction of each effect from the fitted coefficients. Compare these with the plots shown on the Guardian website. Do they agree? Explain.








In [6]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.formula.api import glm




data = pd.read_csv(r"C:\Users\hughd\Desktop\machine learning\brexit.csv")
data['voteBrexit'] = data['voteBrexit'].astype(int)
# create the Y variable
Y = data['voteBrexit']

# create the X variable
x_data = ['abc1', 'medianIncome', 'medianAge', 'withHigherEd', 'notBornUK']
X = data[x_data]

# create a logistic regression model with a binomial distribution and a logit link function
myglm = glm('Y ~ X', data=X, family=sm.families.Binomial()).fit()

print(myglm.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:                  344
Model:                            GLM   Df Residuals:                      338
Model Family:                Binomial   Df Model:                            5
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -123.69
Date:                Thu, 23 Mar 2023   Deviance:                       247.39
Time:                        12:12:40   Pearson chi2:                     401.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1386      0.848     -0.164      0.8

X[0] = 'abc1', X[1] = 'medianIncome', X[2] = 'medianAge', X[3] = 'withHigherEd', X[4] = 'notBornUK'

'medianIncome' and 'withHigherEd' are both negative in direction. Therefore they both voted for remain comparitively to the other variables. This differs from the plots on the guardian website that claim only the 'medianAge' was remain. This is likely due that they used each catagory individually rather than all together.

Question 2)Present the value of each coefficient estimate and calculate the 95% confidence interval. Which input would you say has the strongest effect and why?

In [2]:
import scipy.stats
import numpy as np
zc = scipy.stats.norm.ppf(0.975)


estimate = myglm.params
    
stderr = myglm.bse
   
CI_min = estimate-zc*stderr
CI_max = estimate+zc*stderr
print('coeffs are listed below:')
print(estimate)
print('\nCI values are: ')    
print('Min CI values are: ') 
print(CI_min)
print('\nMin CI values are: ') 
print(CI_max)






coeffs are listed below:
Intercept    -0.138596
X[0]         17.577998
X[1]         -6.385740
X[2]          5.920877
X[3]        -26.744259
X[4]          5.686138
dtype: float64

CI values are: 
Min CI values are: 
Intercept    -1.799992
X[0]         11.871724
X[1]        -10.152204
X[2]          3.164067
X[3]        -33.753456
X[4]          2.151669
dtype: float64

Min CI values are: 
Intercept     1.522799
X[0]         23.284272
X[1]         -2.619275
X[2]          8.677687
X[3]        -19.735062
X[4]          9.220608
dtype: float64


X[0] = 'abc1', X[1] = 'medianIncome', X[2] = 'medianAge', X[3] = 'withHigherEd', X[4] = 'notBornUK'

Using Aic 'withHigherEd' was the most significant variable when comparing which value was most important in determining trends. One reason this might be the case, is the fact that, the coefficient has the largest vale from zero making it the most significant.

Question 3)Using aic, perform a model selection to determine which factors are useful to predict the result of the vote. You can use a ‘greedy’ input selection procedure, as follows: (i) select the best model with 1 input; (ii) fixing that input, select the best two-input model (i.e. try all the other 4 inputs with the one you selected first); (iii) select the best three-input model containing the first two inputs you chose, etc. At each stage evaluate the quality of fit using aic and stop if this gets worse

In [3]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.formula.api import glm


data = pd.read_csv(r"C:\Users\hughd\Desktop\machine learning\brexit.csv")
Y = data['voteBrexit']
x_data = ['abc1', 'medianIncome', 'medianAge', 'withHigherEd', 'notBornUK']


select_cat = []
aic_values = []

for i in range(0, len(x_data)):
    best_aic = float('inf')
    best_cat = None
    
   
    for cat in x_data:
        if cat not in select_cat:
            
            select_cat.append(cat)
            
           
            formula = 'Y ~ ' + ' + '.join(select_cat)
            myglm = glm(formula, data, family=sm.families.Binomial()).fit()
           
            aic = myglm.aic
            
           
            if aic < best_aic:
                best_aic = aic
                best_cat = cat
            
            
            select_cat.pop()
    
   
    if i > 1 and best_aic > aic_values[-1]:
        break
    
   
    select_cat.append(best_cat)
    
  
    aic_values.append(best_aic)
    
    
    print(f"Selected variables: {select_cat}")
    print(f"AIC value: {best_aic:.3f}")
    


Selected variables: ['withHigherEd']
AIC value: 313.560
Selected variables: ['withHigherEd', 'abc1']
AIC value: 286.545
Selected variables: ['withHigherEd', 'abc1', 'medianAge']
AIC value: 271.932
Selected variables: ['withHigherEd', 'abc1', 'medianAge', 'medianIncome']
AIC value: 266.949
Selected variables: ['withHigherEd', 'abc1', 'medianAge', 'medianIncome', 'notBornUK']
AIC value: 259.385


Question 4)Use the Scikit-Learn package to create a decision tree classification model. Visualise your model and intepret the fitted model

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz

tree = DecisionTreeClassifier()
tree.fit(X, Y)


export_graphviz(tree, out_file='dot_data', feature_names=x_data, class_names=['Remain', 'Leave'], filled=True, rounded=True)


!dot -Tpng dot_data -o tree_file1.png


<img src="./tree_file1.png" width=1500, alt="test"/>

For the tree the only factor that was more dominant towards remain was 'medianAge' all the other factors were in favor of leave. The program starts with the most important factor 'withHigherEd' and branches out trying diffrent factors and diffrent combinations of factors until effects of the vote are fully visualised

Question 5)Compare your decision tree model and your logistic regression model. Do they attribute high importance to the same factors? How do you intepret each model to explain the referendum vote?

In [5]:
importance=pd.DataFrame({'variable': ['abc1', 'medianIncome', 'medianAge', 'withHigherEd', 'notBornUK'],'importance value': tree.feature_importances_})
print(importance)

       variable  importance value
0          abc1          0.178019
1  medianIncome          0.143207
2     medianAge          0.132994
3  withHigherEd          0.419795
4     notBornUK          0.125985


Although the tree and the logistic regression model do not agree on the importance of every factor 'withHigherEd' was definetly the most important factor for the vote as both models rated this as the most important by a significant margin. The logistic regression model had two factors that were for remain, 'medianIncome' and 'withHigherEd'. The tree however had both of these factors for leave and only 'medianAge' was for remain. Something that both models do agree on is the order of importance of each factor.

Question 6)Which model would you use if you were explaining the results for a newspaper article, and why? 

Personally I would choose the logistic regression model for a newspaper article. Although the tree method has a visual representation of the results, I think that it is much too confusing to read and interpret, especially as a average reader. Although the logistic method has no visualisation, I think that the direction of the sign is much easier to interpret especially with the explaination that would go with the article. 